In [ ]:
pwd

In [ ]:
tedat = VoxelDataset(datapre.idxTe, dirs.voxel_PN_filepath, params)
teloader = DataLoadertedat, batch_size = batch_size, shuffle=True)



In [ ]:
dic.items()

In [ ]:
"""                  
#non-zero params:

['volFrac', 'CH_11 scaled', 'CH_12 scaled', 'CH_13 scaled',  'CH_22 scaled', 'CH_23 scaled',
'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled', 
'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']
"""
                    



In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat

In [ ]:
import torch

In [ ]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2023/0_data/source_MAT_files'

In [ ]:
dbpath = os.path.join(nbpath, 'dyn_data', 'dynamic_static_database_scaled_DEC23.csv')

In [ ]:
dyndb = pd.read_csv(dbpath)

In [ ]:
dyndb

In [ ]:
# # Completed 14 or 15 March, no need to run it again
# # found the .mat files for the meshes in the dynamic database and saved them as .npy files in the .../dyn_data/voxel_arrays directory
# for topfam in dyndb.topology_family.unique():
#     tfdf = dyndb[dyndb.topology_family == topfam]
#     for ctype in tfdf.cell_type.unique():
#         ctdf = tfdf[tfdf.cell_type == ctype]

#         for i in ctdf.index:
#             matfolder = os.path.join(voxel_path, topfam)
#             dimidx = ctdf.dim_idx[i]
#             matfilename = ctdf.cell_type[i] + '_' + f'{dimidx:.1e}' + '.mat'
#             mat = loadmat(os.path.join(matfolder, matfilename))
#             mesharray = mat[list(mat.keys())[-1]]
#             mesharray = mesharray.astype(np.int8)
#             npyname = topfam +'_' + matfilename[:-4]
#             npypath = os.path.join(voxpath, npyname)
#             np.save(npypath, mesharray)

In [ ]:
voxpath = os.path.join(nbpath, 'dyn_data/voxel_arrays')
stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

In [ ]:
from sklearn.model_selection import train_test_split as TTS

In [ ]:
idxTr, idxRem = TTS(dyndb, stratify = dyndb['topology_family'], random_state=42, train_size = 0.8)
idxVal, idxTe = TTS(idxRem, random_state = 42, test_size=0.5)

In [ ]:
print(idxTr.shape)
print(idxVal.shape)
print(idxTe.shape)

In [ ]:
array_dict = {'Xtr': [], 'Xval': [], 'Xte': []}

for df, array_split in zip([idxTr, idxVal, idxTe], array_dict.keys()):
    arrs= []
    for i in df.index:
        topfam = df.topology_family[i]
        celltype = df.cell_type[i]
        dim_idx = df.dim_idx[i]
        meshname = topfam + '_' + celltype + '_' + f'{dim_idx:.1e}.npy'
        mesh = np.load(os.path.join(voxpath, meshname))
        arrs.append(mesh)

    arrs = np.expand_dims(np.asarray(arrs), axis=1)
    array_dict[array_split] = arrs

In [ ]:
Xtr = array_dict['Xtr']
Xval = array_dict['Xval']
Xte = array_dict['Xte']

In [ ]:
Xtr.shape

In [ ]:
# for i in split_data:
#     print(i.shape)

In [ ]:
data_suffix = 'gsreg'
untruncated = True
cutoff = 750

target_dict = {'Ytr': [], 'Yval': [], 'Yte': []}

# Loop over the splits
for split, tgt in zip([idxTr, idxVal, idxTe], target_dict.keys()):
    # Go through each split row by row, load each dynamic data series, pull selected
    split_data = []
    for idx in split.index:
        dyn_file_name = split['dyn_file_name_original'][idx]
        
        if untruncated == True:
            file_suffix = '_processed_untrunc'
        else:
            file_suffix = '_proct_gaus_btrlp_fftlp'

        dyn_file_full_name = dyn_file_name + file_suffix + '.csv'

        dyn_file_path = os.path.join(stress_ser_path, dyn_file_full_name)

        ser_df = pd.read_csv(dyn_file_path)

        strain = np.expand_dims(ser_df['Strain'][::10].values, axis=0)

        stress_col = f'stress_bottom_{data_suffix}'

        stress = np.expand_dims(ser_df[stress_col][::10].values, axis=0)

        strs_strn_series = torch.tensor(np.concatenate([stress, strain], axis=0))

        split_data.append(strs_strn_series)

    # split_data = split_data

    target_dict[tgt] = split_data

        

In [ ]:
pack_sequence([*target_dict['Yte'][:2]])

In [ ]:
for i in target_dict['Yte']:
    print(i.shape)

In [ ]:
lenslens = []
# Loop over the splits
for split, tgt in zip([idxTr, idxVal, idxTe], target_dict.keys()):
    # Go through each split row by row, load each dynamic data series, pull selected
    split_data = []
    for idx in split.index:
        dyn_file_name = split['dyn_file_name_original'][idx]
        
        if untruncated == True:
            file_suffix = '_processed_untrunc'
        else:
            file_suffix = '_proct_gaus_btrlp_fftlp'

        dyn_file_full_name = dyn_file_name + file_suffix + '.csv'

        dyn_file_path = os.path.join(stress_ser_path, dyn_file_full_name)

        ser_df = pd.read_csv(dyn_file_path)

        lenslens.append(len(ser_df))

In [ ]:
pd.Series(lenslens).describe()

In [ ]:
# {'1A': [0.60,0.65],
# '1B': [0.45, 0.5],
# '1C': [0.15],
# '2A': [0.35, 0.40, 0.45, 0.50],
# '2B': [0.50,],
# '2C': [0.45,],
# '3B': [0.20, 0.25],
# '3C': [0.45,],
# '4A': [0.15,],
# '4B': [0.25,],
# '4C': [0.65,],}



In [ ]:
import torch
from torch.nn.utils.rnn import pack_sequence

# Assuming your data is a list of variable-length sequences
data = [torch.tensor([1, 2, 3]), torch.tensor([4, 5]), torch.tensor([6, 7, 8, 9])]

# Pack the sequences
packed_sequence = pack_sequence(data, enforce_sorted=False)

In [ ]:
packed_sequence

In [ ]:
csv = os.path.join(stress_ser_path, 'RhoCross_0p14_sr1000_proct_gaus_btrlp_fftlp.csv')

In [ ]:
csvdf = pd.read_csv(csv)

In [ ]:
stress = np.expand_dims(csvdf['Strain'][::10].values, axis=0)

In [ ]:
strain = np.expand_dims(csvdf['stress_bottom_gsreg'][::10].values, axis=0)

In [ ]:
np.concatenate([stress, strain], axis=0).shape